In [17]:
import numpy as np

# np.zeros((5,4))
np.random.randint(1,7,(5,4))

array([[4, 6, 3, 4],
       [6, 1, 6, 5],
       [1, 3, 5, 1],
       [4, 3, 4, 2],
       [3, 1, 3, 2]])

In [4]:
from collections import Counter

c = Counter([1,2,3])
c[4]

0

In [20]:
import sys
sys.maxsize

-9223372036854775807

## Cloud

In [ ]:
#Read many part files from AWS s3

import boto3, io, s3fs
from multiprocessing import Pool

## Connections

# conn = connect(
#     aws_access_key_id='YOUR_ACCESS_KEY_ID',
#     aws_secret_access_key='YOUR_SECRET_ACCESS_KEY',
#     s3_staging_dir='s3://bucket/staging/',
#     region_name='us-east-1')

# cursor = connect(
#     aws_access_key_id='YOUR_ACCESS_KEY_ID',
#     aws_secret_access_key='YOUR_SECRET_ACCESS_KEY',
#     s3_staging_dir='s3://bucket/staging/',
#     region_name='us-east-1').cursor()

## --------------------------------------------------
## Pandas batch style based data fetch (Error: Query times out for large datasets)

# def read_data_from_athena(db_name, table_name):
#     query = "SELECT * FROM {}.{}".format(db_name, table_name) 
#     cur_df = pd.read_sql(query, conn)
#     return cur_df

# raw_training_df = read_data_from_athena(db_name, table_name)

## --------------------------------------------------
## Cursor based data fetch (Error: No result set for large datasets)

# def get_cursor_iterator(db_name, table_name):
#     query = "SELECT * FROM {}.{} limit 10".format(db_name, table_name)
#     print(query)
#     for row in cursor:
#         print(row)
#         break

# get_cursor_iterator(db_name, table_name)

## --------------------------------------------------
def serial_read_part_files_to_df(bucket_name, parent_key):
    s3 = boto3.resource('s3')
    bucket = s3.Bucket(bucket_name)
    prefix_objs = bucket.objects.filter(Prefix=parent_key)
    sub_df = []
    for obj in prefix_objs:
        file_key = obj.key
        print(file_key)
        cur_df = pd.read_csv('s3://'+bucket_name+'/'+file_key, compression='gzip', header=0, sep='|', quotechar='"')
        cur_df = cur_df[valid_cols]
        sub_df.append(cur_df)
    return pd.concat(sub_df)

def get_data(bucket_name, file_key, valid_cols):
    print(file_key)
    cur_df = pd.read_csv('s3://'+bucket_name+'/'+file_key, compression='gzip', header=0, sep='|', quotechar='"')
    cur_df = cur_df[valid_cols]
    return cur_df

class GET_DATA(): #Helper class to invoke get_data() as local lambda's cannot be pickled so cannot be used with pool.map. functools.partial can also be used 
    def __init__(self, bucket_name, valid_cols):
        self.bucket_name = bucket_name
        self.valid_cols = valid_cols
    def __call__(self, file_key):
        return get_data(self.bucket_name, file_key, self.valid_cols)

def parallel_read_part_files_to_df(bucket_name, parent_key):
    s3 = boto3.resource('s3')
    bucket = s3.Bucket(bucket_name)
    prefix_objs = bucket.objects.filter(Prefix=parent_key)
    file_keys = tuple(map(lambda x : x.key, prefix_objs))
    agents, chunksize = 5, 25
    GD = GET_DATA(bucket_name, valid_cols)
    with Pool(processes=agents) as pool:
        sub_df = pool.map(GD, file_keys, chunksize)
    return pd.concat(sub_df)


## Docker